# Multiple Linear Regression 

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from sklearn.linear_model import LinearRegression

In [9]:
path='C:/Users/Mendes/Desktop/The Data Science Course 2021 - All Resources/Part_5_Advanced_Statistical_Methods_(Machine_Learning)/S34_L212/'
data = pd.read_csv(path+'1.02. Multiple linear regression.csv')
data.head()

,SAT,"Rand 1,2,3",GPA
0,1714,1,2.40
1,1664,3,2.52
2,1760,3,2.54
3,1685,3,2.74
4,1693,2,2.83


In [10]:
data.describe()

,SAT,"Rand 1,2,3",GPA
count,84.000000,84.000000,84.000000
mean,1845.273810,2.059524,3.330238
std,104.530661,0.855192,0.271617
min,1634.000000,1.000000,2.400000
25%,1772.000000,1.000000,3.190000
50%,1846.000000,2.000000,3.380000
75%,1934.000000,3.000000,3.502500
max,2050.000000,3.000000,3.810000


## Create the multiple linear regression

In [11]:
# Declare the dependet and indepedent variables
x=data[['SAT','Rand 1,2,3']]
y=data['GPA']

In [7]:
# Regression itself
reg=LinearRegression()
reg.fit(x,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [9]:
reg.coef_  #corresponds to SAT and Rand 1,2,3 

array([ 0.00165354, -0.00826982])

In [10]:
reg.intercept_

0.29603261264909486

In [16]:
# Calculating the R-squared
reg.score(x,y)
r2=reg.score(x,y)

### Formula for Adjusted R^2

$R^2_{adj.} = 1 - (1-R^2)*\frac{n-1}{n-p-1}$

In [20]:
x.shape #(n= # of observations, p = # of predictors)
n=x.shape[0]
p=x.shape[1]

In [24]:
r2_adj=1-(1-r2)*((n-1)/(n-p-1))

In [26]:
print(r2_adj)
#Adj. R^2 < R^2, therefore one or more of the predictors have little or no explanatory power

0.39203134825134023


# Feature selection with F-regression

In [30]:
# Feature selection simplifies models, improves speed and prevents a series of unwanted issues arising from having 
# too many features

In [31]:
from sklearn.feature_selection import f_regression
# F-regression creates simple linear regressions of each feature and the dependet variable

In [32]:
f_regression(x,y)  #F-statistcs and corresponding p-values

(array([56.04804786,  0.17558437]), array([7.19951844e-11, 6.76291372e-01]))

In [34]:
p_values=f_regression(x,y)[1]
p_values

array([7.19951844e-11, 6.76291372e-01])

In [36]:
p_values.round(3)   #refers to 'SAT' and 'Rand 1, 2, 3'. SAT is usefull and Rand is useless in this case. 

array([0.   , 0.676])

In [38]:
# Note: these are the univariate p-values reached from simple linear models. They do not reflect the interconnection of 
# the features in our multiple linear regression.

# Creating a summary table

In [44]:
#reg_summary = pd.DataFrame (data=['SAT','Rand 1,2,3'], columns=['Features'])
reg_summary = pd.DataFrame (data=x.columns.values, columns=['Features'])
reg_summary

,Features
0,SAT
1,"Rand 1,2,3"


In [45]:
reg_summary['Coefficients']=reg.coef_
reg_summary['p-values']=p_values.round(3)

In [49]:
reg_summary  #P-values are one of the best ways to determine if a variable is redundant, but they provide no information 
# whatsoever about how useful a variable is.

,Features,Coefficients,p-values
0,SAT,0.001654,0.000
1,"Rand 1,2,3",-0.008270,0.676


# Standardization

In [12]:
from sklearn.preprocessing import StandardScaler

In [13]:
scaler=StandardScaler()   #scaler will be used to subtract the mean and divide by the standard deviation

In [14]:
scaler.fit(x)  #calculates and stores the mean and std deviation of each feature
x_scaled=scaler.transform(x)
#new_data_scaled=scaler.transform(new_data)

In [15]:
# Regression with scaled features
reg=LinearRegression()
reg.fit(x_scaled,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [16]:
reg.coef_

array([ 0.17181389, -0.00703007])

In [17]:
reg.intercept_

3.330238095238095

In [27]:
# Creating a summary table
reg_summary=pd.DataFrame([['Bias'],['SAT'],['Rand 1,2,3']],columns=['Features'])
reg_summary['Weights']=reg.intercept_,reg.coef_[0],reg.coef_[1]
reg_summary  #The bigger the weight, the bigger the impact of the feature. The closer a weight is to 0, the smaller its impact.

,Features,Weights
0,Bias,3.330238
1,SAT,0.171814
2,"Rand 1,2,3",-0.007030


In [33]:
#Making predictions with the standardized coefficients (weights)
new_data=pd.DataFrame(data=[[1700,2],[1800,1]],columns=['SAT','Rand 1,2,3'])  #the new data frame should be arranged and standardized in the same way
new_data_scaled=scaler.transform(new_data)
new_data_scaled

array([[-1.39811928, -0.07002087],
       [-0.43571643, -1.24637147]])

In [34]:
reg.predict(new_data_scaled)

array([3.09051403, 3.26413803])

In [35]:
#What if we removed the 'Random 1,2,3' variable?
reg_simple=LinearRegression()
x_simple_matrix=x_scaled[:,0].reshape(-1,1)
reg_simple.fit(x_simple_matrix,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)